In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, Model
import os
tf.random.set_seed(1111)
np.random.seed(1111)
pred_list = list()
featureslist =['AQI_','C_PM25_1H','I_PM25_24H_MV']
days = {'51010901001_5': 168, '51010901001_12': 183, '51010801001_5': 168, '51010801001_12': 186, '51010501001_5': 168, '51010501001_12': 180, '51010501002_5': 168, '51010501002_12': 186, '51010401001_5': 168, '51010401001_12': 185, '51010101001_5': 168, '51010101001_12': 183, '51010601002_5': 168, '51010601002_12': 182, '51011201004_5': 168, '51011201004_12': 186}
path = "train_cls"
files = os.listdir(path)
units = 16

pred_list = list()
data = pd.read_csv(path+"\\"+addre)
mn = list(data['MN_'])
mnname = mn[0]
datatime = list(data['DATA_TIME_'])
del data['DATA_TIME_']
if 'UPDATE_TIME_' in data.columns:
    del data['UPDATE_TIME_']
del data['MN_']
data = pd.get_dummies(data)
# 删除全空特征
for col in list(data.columns):
    if data[col].isnull().all():
        del data[col]
# 填补缺失值
columns = list(data.columns)
for c in columns:
    data[c] = data[c].fillna(method='pad')

data['DATA_TIME_'] = datatime
timepoint = 0
for i in range(len(data.values)):
    if data['DATA_TIME_'][i] == '2017/5/31 23:00:00' or data['DATA_TIME_'][i] == '2017/5/31 23:00' or(data['DATA_TIME_'][i] == '2017/10/30 13:00:00' and mnname==51011201004):
        timepoint = i
        break
length1 = timepoint +1 # 上半年的数据总长度


# 预测上半年的AQI
tw_y = days[str(mnname)+"_5"]
tw_x = 2*tw_y
preday1 = (length1-tw_y)%tw_y
data1 = data[preday1:length1]
pred1_x = data[length1-tw_y:length1]
data1_x,data1_y,pred1_x = pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
for i in range(len(data1)-(tw_x+tw_y)):
    data1_x = pd.concat([data1_x, data1[featureslist][i:i+tw_x]])
    data1_y = pd.concat([data1_y, data1['AQI_'][i+tw_x:i+tw_x+tw_y]])
pred1_x = data1[featureslist][len(data1)-(tw_x+tw_y):len(data1)-tw_y]
data1_x1 = np.array(data1_x)
data1_x1 = data1_x1.reshape(-1,tw_x,len(featureslist))
data1_y1 = np.array(data1_y)
data1_y1 = data1_y1.reshape(-1,tw_y,1)
pred1_x1 = np.array(pred1_x)
pred1_x1 = pred1_x1.reshape(-1,tw_x,len(featureslist))

In [ ]:
data1_x1.shape,data1_y1.shape

In [12]:
inputs = keras.Input(shape=(data1_x1.shape[1], data1_x1.shape[2]))
lstm = layers.LSTM(units=units, input_shape=(data1_x1.shape[1], data1_x1.shape[2]))(inputs)
fc1 = layers.Dense(tw_y*2)(lstm)
fc2 = layers.Dense(tw_y)(fc1)
class_models = keras.Model(inputs=[inputs], outputs=[fc2])
# class_models.summary()
class_models.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
history1 = class_models.fit([data1_x1], data1_y1, epochs=100,batch_size=512, verbose=2)
predict1 = class_models.predict([pred1_x1])
pred_list.append(predict1)

Train on 2856 samples
Epoch 1/100
2856/2856 - 7s - loss: 10893.8881 - accuracy: 0.0056
Epoch 2/100
2856/2856 - 3s - loss: 10821.0470 - accuracy: 0.0056
Epoch 3/100
2856/2856 - 3s - loss: 10714.4633 - accuracy: 0.0056
Epoch 4/100
2856/2856 - 3s - loss: 10555.8185 - accuracy: 0.0067
Epoch 5/100
2856/2856 - 3s - loss: 10328.5742 - accuracy: 0.0060
Epoch 6/100
2856/2856 - 3s - loss: 10006.7637 - accuracy: 0.0060
Epoch 7/100
2856/2856 - 3s - loss: 9598.8230 - accuracy: 0.0060
Epoch 8/100
2856/2856 - 3s - loss: 9100.6703 - accuracy: 0.0056
Epoch 9/100
2856/2856 - 3s - loss: 8491.4236 - accuracy: 0.0056
Epoch 10/100
2856/2856 - 3s - loss: 7769.1314 - accuracy: 0.0056
Epoch 11/100
2856/2856 - 3s - loss: 7015.8530 - accuracy: 0.0056
Epoch 12/100
2856/2856 - 3s - loss: 6253.8402 - accuracy: 0.0056
Epoch 13/100
2856/2856 - 3s - loss: 5533.0262 - accuracy: 0.0056
Epoch 14/100
2856/2856 - 3s - loss: 4872.0577 - accuracy: 0.0056
Epoch 15/100
2856/2856 - 3s - loss: 4246.9523 - accuracy: 0.0056
Epoch 

In [13]:
inputs = keras.Input(shape=(data1_x1.shape[1], data1_x1.shape[2]))
lstm = layers.GRU(units=units, input_shape=(data1_x1.shape[1], data1_x1.shape[2]))(inputs)
fc1 = layers.Dense(tw_y*2)(lstm)
fc2 = layers.Dense(tw_y)(fc1)
class_models = keras.Model(inputs=[inputs], outputs=[fc2])
# class_models.summary()
class_models.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
history1 = class_models.fit([data1_x1], data1_y1, epochs=100,batch_size=512, verbose=2)
predict1 = class_models.predict([pred1_x1])
pred_list.append(predict1)

Train on 2856 samples
Epoch 1/100
2856/2856 - 8s - loss: 10878.4292 - accuracy: 0.0074
Epoch 2/100
2856/2856 - 4s - loss: 10774.1805 - accuracy: 0.0063
Epoch 3/100
2856/2856 - 4s - loss: 10619.1325 - accuracy: 0.0067
Epoch 4/100
2856/2856 - 4s - loss: 10392.2639 - accuracy: 0.0060
Epoch 5/100
2856/2856 - 4s - loss: 10080.8863 - accuracy: 0.0070
Epoch 6/100
2856/2856 - 4s - loss: 9671.1842 - accuracy: 0.0081
Epoch 7/100
2856/2856 - 4s - loss: 9155.3699 - accuracy: 0.0081
Epoch 8/100
2856/2856 - 4s - loss: 8557.8840 - accuracy: 0.0081
Epoch 9/100
2856/2856 - 4s - loss: 7881.6164 - accuracy: 0.0077
Epoch 10/100
2856/2856 - 4s - loss: 7128.6951 - accuracy: 0.0074
Epoch 11/100
2856/2856 - 4s - loss: 6348.9400 - accuracy: 0.0067
Epoch 12/100
2856/2856 - 5s - loss: 5609.4997 - accuracy: 0.0060
Epoch 13/100
2856/2856 - 5s - loss: 4921.9163 - accuracy: 0.0056
Epoch 14/100
2856/2856 - 4s - loss: 4317.1848 - accuracy: 0.0056
Epoch 15/100
2856/2856 - 4s - loss: 3819.2958 - accuracy: 0.0056
Epoch 1

In [14]:
inputs = keras.Input(shape=(data1_x1.shape[1], data1_x1.shape[2]))
lstm = layers.recurrent.SimpleRNN(units=units, input_shape=(data1_x1.shape[1], data1_x1.shape[2]))(inputs)
fc1 = layers.Dense(tw_y*2)(lstm)
fc2 = layers.Dense(tw_y)(fc1)
class_models = keras.Model(inputs=[inputs], outputs=[fc2])
# class_models.summary()
class_models.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
history1 = class_models.fit([data1_x1], data1_y1, epochs=100,batch_size=512, verbose=2)
predict1 = class_models.predict([pred1_x1])
pred_list.append(predict1)

AttributeError: module 'tensorflow_core.keras.layers' has no attribute 'recurrent'

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.epoch, history.history.get('accuracy'), label='train')
plt.plot(history.epoch, history.history.get('val_accuracy'), label='test')
plt.legend()
plt.title("The accuracy of train and test")
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.show()

In [7]:
a = pred_list[0][0]

[61.747303,
 61.82827,
 61.643433,
 61.567528,
 61.362312,
 61.49442,
 61.430717,
 61.494904,
 61.383163,
 61.114697,
 61.012943,
 61.18275,
 61.040546,
 61.133217,
 60.886387,
 60.64818,
 60.84329,
 60.821278,
 60.719032,
 60.626537,
 60.788574,
 60.549057,
 60.56914,
 60.796688,
 60.647057,
 60.836243,
 60.621548,
 60.64766,
 60.346626,
 60.18028,
 60.353264,
 60.45632,
 60.204468,
 60.156677,
 60.199333,
 60.325768,
 59.91805,
 59.95473,
 60.186718,
 60.13291,
 59.930664,
 60.104736,
 60.254345,
 59.9462,
 60.260284,
 60.07094,
 60.185146,
 59.867264,
 59.88892,
 59.69724,
 59.78014,
 59.686455,
 59.621418,
 59.693165,
 59.53849,
 59.454678,
 59.617825,
 59.293137,
 59.370678,
 59.50128,
 59.478024,
 59.338184,
 59.317654,
 59.444492,
 59.270424,
 59.231285,
 59.31883,
 59.17599,
 59.251907,
 59.49783,
 59.341515,
 59.33526,
 59.139816,
 59.264362,
 59.28834,
 58.833694,
 59.132084,
 59.031364,
 59.004883,
 58.83159,
 58.92728,
 58.84275,
 58.897366,
 58.704147,
 58.759396,
 58.7124